In [51]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [38]:
def readDict(filename, sep):
    with open(filename, "r") as f:
        value_dic = {}
        dtype_dic = {}
        stype_dic = {}
        label_dic = {}
        for line in f:
            key, values = line.split(sep)
            values = values.split(',')
            if 'value' in values[1]:
                value_dic[key] = int(values[0])
            elif "dtype" in values[1]:
                dtype_dic[key] = int(values[0])
            elif "stype" in values[1]:
                stype_dic[key] = int(values[0])
            elif "label" in values[1]:
                label_dic[key] = int(values[0])
            else:
                print('wow')
            #dict[values[0][2:-1]] = {int(x[:-3]) for x in values[1:len(values)]}
        return(value_dic, dtype_dic, stype_dic, label_dic)

In [48]:
cols = ['col_name','# unique values','# missing values','% missing values','base type','semantic type','# valid', '# null', '# invalid']
df_cols = pd.DataFrame(columns=cols)
outputs = os.listdir('./output')
outputs = [f for f in outputs if f[-3:]=='out']
cnt = 0
for file in outputs:
    df_cols.loc[cnt, 'col_name'] = file[:-4]
    value_dic, dtype_dic, stype_dic, label_dic = readDict('./output/'+file, '\t')
    df_cols.loc[cnt, '# unique values'] = len(value_dic)
    if value_dic.get('NULL'):
        df_cols.loc[cnt, '# missing values'] = int(value_dic.get('NULL'))
        df_cols.loc[cnt, '% missing values'] = str(round(float(value_dic.get('NULL') / 5101231 * 100), 2))+'%'
    else:
        df_cols.loc[cnt, '# missing values'] = 0
        df_cols.loc[cnt, '% missing values'] = '0.0%'
    df_cols.loc[cnt, 'base type'] = ','.join(list(dtype_dic.keys()))
    df_cols.loc[cnt, 'semantic type'] = ','.join(list(stype_dic.keys()))
    df_cols.loc[cnt, '# valid'] = label_dic['VALID'] if 'VALID' in label_dic else 0
    df_cols.loc[cnt, '# null'] = label_dic['NULL'] if 'NULL' in label_dic else 0
    df_cols.loc[cnt, '# invalid'] = label_dic['INVALID'] if 'INVALID' in label_dic else 0
    cnt = cnt + 1
    #print(file)

In [54]:
df_cols

,col_name,# unique values,# missing values,% missing values,base type,semantic type,# valid,# null,# invalid
0,ADDR_PCT_CD,78,390,0.01%,"string,int",precinct,5100832,390,0
1,BORO_NM,6,463,0.01%,string,borough,5100759,463,0
2,CMPLNT_FR_DT,6372,655,0.01%,"datetime,string",date,5100567,655,0
3,CMPLNT_FR_TM,1443,48,0.0%,"string,timestamp",time,5100271,48,903
4,CMPLNT_TO_DT,4828,1391476,27.28%,"datetime,string",date,3709746,1391476,0
5,CMPLNT_TO_TM,1442,1387783,27.2%,"string,timestamp",time,3712063,1387783,1376
6,CRM_ATPT_CPTD_CD,3,7,0.0%,string,flag,5101215,7,0
7,HADEVELOPT,279,4848018,95.04%,string,address,253204,4848018,0
8,JURIS_DESC,25,0,0.0%,string,jurisdiction,5101222,0,0
9,KY_CD,74,0,0.0%,int,code,5101222,0,0
